## 1. セットアップ

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path
from sklearn.decomposition import PCA

print("[OK] ライブラリのセットアップ完了")

## 2. オープンデータセット（BBC News）の読み込み

In [ ]:
# BBC News Dataset をsklearnから取得
from sklearn.datasets import fetch_20newsgroups
import warnings
warnings.filterwarnings('ignore')

print("[INFO] テキストデータセットを読み込み中...")

# 20 Newsgroupsデータセット（大規模テキスト分類）
categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.med',
    'talk.politics.guns'
]

newsgroups = fetch_20newsgroups(
    subset='train',
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=('headers', 'footers', 'quotes')
)

# DataFrameに変換
df = pd.DataFrame({
    'text': newsgroups.data,
    'category': [categories[i] for i in newsgroups.target],
    'category_id': newsgroups.target
})

# テキストの前処理（最初の500文字に制限）
df['text'] = df['text'].str[:500]

print(f"[OK] データ読み込み完了: {len(df)} 件")
print(f"\nカテゴリ分布:")
print(df['category'].value_counts())
print(f"\nサンプルテキスト:")
print(df['text'].iloc[0][:200])

## 3. テキスト前処理

In [ ]:
from preprocessor.text_cleaner import TextCleaner

# テキストクリーニング
df['cleaned_text'] = df['text'].apply(TextCleaner.clean)

print("[OK] テキストクリーニング完了")
print(f"\n処理前後の比較:")
print(f"処理前: {df['text'].iloc[0][:100]}...")
print(f"処理後: {df['cleaned_text'].iloc[0][:100]}...")

## 4. トークン化

In [ ]:
from preprocessor.tokenizer import Tokenizer

# 簡易トークン化（空白分割）
# 注: 日本語テキストではないため、英語の空白分割を使用
df['tokens'] = df['cleaned_text'].apply(lambda x: x.lower().split())

print("[OK] トークン化完了")
print(f"\nサンプルトークン:")
print(df['tokens'].iloc[0][:20])

## 5. ベクトル化（Sentence-Transformers）

In [ ]:
from analysis.vectorizer import TextVectorizer

print("[INFO] ベクトル化を実行中...")

# ベクトル化（最初の300件を使用して計算時間を短縮）
vectorizer = TextVectorizer(use_mock=True)

# サンプル削減
df_sample = df.head(300).copy()
texts = df_sample['cleaned_text'].tolist()
embeddings = vectorizer.encode(texts, normalize=True)

print(f"[OK] ベクトル化完了")
print(f"埋め込み次元: {embeddings.shape[1]}")
print(f"ドキュメント数: {embeddings.shape[0]}")
print(f"サンプル埋め込み（最初の10次元）: {embeddings[0][:10]}")

## 6. クラスタリング（HDBSCAN）

In [ ]:
from analysis.clustering import TopicClusterer

print("[INFO] HDBSCAN クラスタリング実行中...")

clusterer = TopicClusterer(min_cluster_size=5, min_samples=2)
clusterer.fit(embeddings)

df_sample['cluster'] = clusterer.get_cluster_labels()

print(f"[OK] クラスタリング完了")
print(f"検出クラスタ数: {clusterer.n_clusters}")
print(f"\nクラスタ分布:")
print(df_sample['cluster'].value_counts().sort_index())

# クラスタ統計
stats = clusterer.get_cluster_stats()
print(f"\nクラスタ統計情報:")
for cluster_id in sorted([k for k in stats.keys() if k != 'noise'])[:5]:
    stat = stats[cluster_id]
    print(f"  Cluster {cluster_id}: {stat['size']} items, mean_distance={stat['mean_distance']:.3f}")

## 7. 次元圧縮（PCA）と可視化

In [ ]:
print("[INFO] PCA 次元圧縮中...")

# PCA で2次元に圧縮
pca = PCA(n_components=2, random_state=42)
embeddings_2d = pca.fit_transform(embeddings)

print(f"[OK] PCA 圧縮完了")
print(f"説明分散（PC1+PC2）: {pca.explained_variance_ratio_.sum():.2%}")
print(f"PC1説明分散: {pca.explained_variance_ratio_[0]:.2%}")
print(f"PC2説明分散: {pca.explained_variance_ratio_[1]:.2%}")
print(f"2D埋め込み形状: {embeddings_2d.shape}")

## 8. インタラクティブな散布図表示

In [ ]:
# データフレーム作成
plot_data = pd.DataFrame({
    'x': embeddings_2d[:, 0],
    'y': embeddings_2d[:, 1],
    'text': df_sample['cleaned_text'].values,
    'category': df_sample['category'].values,
    'cluster': df_sample['cluster'].astype(str).values
})

# クラスタ色分けの散布図
fig = px.scatter(
    plot_data,
    x='x',
    y='y',
    color='cluster',
    hover_data={'text': ':<100', 'category': True},
    title='Document Clustering Visualization (PCA + HDBSCAN)',
    labels={'x': 'PC1', 'y': 'PC2'},
    height=800,
    width=1200,
    hover_name='category'
)

fig.update_traces(
    marker=dict(size=8, opacity=0.7, line=dict(width=0.5, color='white'))
)

fig.update_layout(
    hovermode='closest',
    plot_bgcolor='#f8f9fa',
    font=dict(family="Arial, sans-serif", size=12)
)

fig.show()

print(f"[OK] インタラクティブ散布図を表示しました")
print(f"\n💡 Tips: ホバーするとテキストプレビューが表示されます")

## 9. カテゴリ別可視化

In [ ]:
# カテゴリ色分けの散布図
fig2 = px.scatter(
    plot_data,
    x='x',
    y='y',
    color='category',
    hover_data={'text': ':<100'},
    title='Document Clustering by Category',
    labels={'x': 'PC1', 'y': 'PC2'},
    height=800,
    width=1200,
    hover_name='category'
)

fig2.update_traces(
    marker=dict(size=8, opacity=0.7, line=dict(width=0.5, color='white'))
)

fig2.update_layout(
    hovermode='closest',
    plot_bgcolor='#f8f9fa'
)

fig2.show()

print(f"[OK] カテゴリ別可視化を表示しました")

## 10. 共起ネットワーク分析

In [ ]:
from analysis.cooccurrence import CooccurrenceNetwork

print("[INFO] 共起ネットワーク構築中...")

tokenized_docs = df_sample['tokens'].tolist()
network = CooccurrenceNetwork(window_size=5, min_frequency=2)
graph = network.build_network(tokenized_docs)

print(f"[OK] ネットワーク構築完了")
print(f"ノード数（ユニーク単語）: {graph.number_of_nodes()}")
print(f"エッジ数（共起関係）: {graph.number_of_edges()}")

# トップキーワード
top_keywords = network.get_keywords_by_frequency(top_n=15)
print(f"\n出現頻度トップ15キーワード:")
print(top_keywords.to_string(index=False))

## 11. 共起エッジ分析

In [ ]:
# トップ共起エッジ
top_edges = network.get_top_edges(top_n=15)
print("共起頻度トップ15ペア:")
print(top_edges.to_string(index=False))

## 12. ネットワーク中心性指標

In [ ]:
# 中心性指標を計算
degree = network.get_node_degree()
strength = network.get_node_strength()
betweenness = network.get_betweenness_centrality()

print("=" * 60)
print("ネットワーク中心性指標")
print("=" * 60)

print(f"\n【次数中心性 (Degree Centrality)】")
print("ノードが直結している他のノード数:")
top_degree = sorted(degree.items(), key=lambda x: x[1], reverse=True)[:10]
for word, deg in top_degree:
    print(f"  {word:20s} : {deg:3d}")

print(f"\n【重み付き次数 (Strength)】")
print("共起頻度を加味した接続度:")
top_strength = sorted(strength.items(), key=lambda x: x[1], reverse=True)[:10]
for word, str_val in top_strength:
    print(f"  {word:20s} : {str_val:8.1f}")

print(f"\n【媒介中心性 (Betweenness Centrality)】")
print("ノード間の最短経路上に出現する頻度:")
top_betweenness = sorted(betweenness.items(), key=lambda x: x[1], reverse=True)[:10]
for word, bet_val in top_betweenness:
    print(f"  {word:20s} : {bet_val:.4f}")

## 13. クラスタ要約

In [ ]:
# クラスタ要約
summary = clusterer.get_cluster_summary(texts, top_n=3)

print("\n" + "=" * 70)
print("クラスタ別テキスト要約")
print("=" * 70)

for cluster_id in sorted([k for k in summary.keys()]):
    if cluster_id == 'noise':
        continue
    info = summary[cluster_id]
    print(f"\n【Cluster {cluster_id}】")
    print(f"  サイズ: {info['size']} ドキュメント")
    print(f"  代表的なテキスト:")
    for i, text in enumerate(info['representative_texts'][:2], 1):
        preview = text[:70] + "..." if len(text) > 70 else text
        print(f"    {i}. {preview}")

## 14. 統計サマリー

In [ ]:
print("\n" + "=" * 70)
print("分析結果サマリー")
print("=" * 70)

print(f"\n📊 データ処理:")
print(f"  - 元のデータセット: {len(df)} ドキュメント")
print(f"  - 分析対象: {len(df_sample)} ドキュメント")
print(f"  - カテゴリ数: {df['category'].nunique()}")

print(f"\n🔤 ベクトル化:")
print(f"  - 埋め込みモデル: Sentence-Transformers")
print(f"  - 埋め込み次元: {embeddings.shape[1]}")
print(f"  - モード: Mock (テスト用)")

print(f"\n🎯 クラスタリング (HDBSCAN):")
print(f"  - 検出クラスタ数: {clusterer.n_clusters}")
print(f"  - ノイズポイント: {(df_sample['cluster'] == -1).sum()}")
if clusterer.get_silhouette_score() is not None:
    print(f"  - シルエット係数: {clusterer.get_silhouette_score():.3f}")

print(f"\n📈 次元圧縮 (PCA):")
print(f"  - 圧縮法: PCA (UMAPはLLVM互換性問題のためPCAを使用)")
print(f"  - 最終次元: 2D")
print(f"  - 説明分散: {pca.explained_variance_ratio_.sum():.2%}")

print(f"\n🌐 共起ネットワーク分析:")
print(f"  - ユニーク単語数: {graph.number_of_nodes()}")
print(f"  - 共起関係（エッジ）: {graph.number_of_edges()}")
print(f"  - 最頻出単語: {top_keywords.iloc[0]['word']} ({top_keywords.iloc[0]['frequency']} 回)")

print(f"\n✨ 可視化:")
print(f"  - インタラクティブ散布図: Plotly")
print(f"  - ホバー機能: あり（テキストプレビュー）")
print(f"  - カテゴリ別表示: あり")

print("\n" + "=" * 70)
print("✅ 分析完了！")
print("=" * 70)